In [ ]:
from os import listdir
import numpy as np
from numpy import expand_dims, zeros, ones, vstack, asarray, savez_compressed, load 
from numpy.random import randn, randint
from keras.preprocessing.image import img_to_array, load_img
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout
from matplotlib import pyplot
from keras.models import load_model
from numpy.random import randn
from matplotlib import pyplot

In [ ]:
def define_discriminator(in_shape=(256,256,3)):
    model = Sequential()
    model.add(Conv2D(4, 4, padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(8, 4, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(16, 4, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(32, 4, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(64, 4, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, 4, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(256, 4, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(512, 4, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

def define_generator(latent_dim):
    model = Sequential()
    n_nodes = 256 * 8 * 8
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((8, 8, 256)))
    model.add(Conv2DTranspose(512, 4, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(256, 4, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(128, 4, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(64, 4, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(32, 4, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(3, (3,3), activation='sigmoid', padding='same'))
    return model

def define_gan(g_model, d_model):
    d_model.trainable = False
    model = Sequential()
    model.add(g_model)
    model.add(d_model)
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [ ]:
image_dict = dict()
path = 'data/tm0050/'
size = (256,256)

for i in range(231):
    pixel1 = img_to_array(load_img(path+str(i)+'.jpg', target_size=size))
    pixel2 = img_to_array(load_img(path+str(i+1)+'.jpg', target_size=size))
    image_dict[i] = pixel1, pixel2

In [ ]:
# Load original scans based on the imposed scan(key in dict)
def load_real_samples(key):
    X1 = image_dict[key][0] / 255.0
    X1 = np.expand_dims(X1, axis=0)
    X2 = image_dict[key][1] / 255.0
    X2 = np.expand_dims(X2, axis=0)
    X = vstack((X1, X2))
    y = ones((2,1))
    return X, y

# Run an image through MLP to get latent representation.
def generate_latent_representation(latent_dim, image_shape=(256,256,3)):
    model = Sequential()
    model.add(Conv2D(64, 3, padding='same', input_shape=image_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, 3, strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(256, 3, strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2)) 
    model.add(Flatten())
    model.add(Dense(latent_dim))
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

# Generate two fake samples
def generate_fake_samples(image, g_model, MLP): 
    X = image / 255.0
    X = expand_dims(X, 0)
    X = MLP.predict(X)
    X = g_model.predict(X)
    y = zeros(1) 
    return X, y

In [ ]:
def save_plot(X1, X2, epoch):
    # plot images
    for i in range(2 * 2):
        # define subplot
        pyplot.subplot(2, 2, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(X1[i, :, :, 0], cmap='gray_r')
        pyplot.imshow(X2[i, :, :, 0], cmap='gray_r')
    # save plot to file
    filename = 'generated_plot_e%03d.jpg' % (epoch+1)
    pyplot.savefig(filename)
    pyplot.close()

def train(MLP, g1_model, g2_model, d_model, gan1_model, gan2_model, path, size=256, n_epochs=100):
    for i in range(n_epochs):
        for filename in listdir(path):
            j = filename.find(".")
            name = filename[:j]
            image = load_img(path+filename, target_size=(size,size))
            image = img_to_array(image)
            
            X_real, y_real = load_real_samples(int(name))
            X1_fake, y1_fake = generate_fake_samples(image, g1_model, MLP)
            X2_fake, y2_fake = generate_fake_samples(image, g2_model, MLP)
            X_fake, y_fake = vstack((X1_fake, X2_fake)), vstack((y1_fake, y2_fake))
            X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
            d_loss, _ = d_model.train_on_batch(X, y)  
            
            X_gan = image / 255.0
            X_gan = expand_dims(X_gan, 0)
            X_gan = MLP.predict(X_gan)
            y_gan = ones((4,1))
            g_loss1 = gan1_model.train_on_batch(X_gan, y_gan)
            g_loss2 = gan2_model.train_on_batch(X_gan, y_gan)
        if (i+1) % 100 == 0:
            print('>%d, d=%.3f, d=%.3f, g=%.3f' % (i+1, d_loss, g_loss1, g_loss2))
            save_plot(X1_fake, X2_fake, i)
        if (i+1) % 500 == 0:
            fname1 = 'generator_model_%03d.h5' % (i + 1)
            fname2 = 'generator_model_%03d.h5' % (i + 1)
            g1_model.save(fname1)
            g2_model.save(fname2)

In [ ]:
latent_dim = 100
g1_model = define_generator(latent_dim)
g2_model = define_generator(latent_dim)
d_model = define_discriminator()
gan1_model = define_gan(g1_model, d_model)
gan2_model = define_gan(g2_model, d_model)
path = 'data/impo0050/'
MLP = generate_latent_representation(latent_dim)

train(MLP, g1_model, g2_model, d_model, gan1_model, gan2_model, path, n_epochs=1000)